In [1]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 45.9 MB/s 
     |████████████████████████████████| 180 kB 39.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7d0ddf62ce7bc1f2b5eeb6f5aa7e8a68fd0113466b0d0f6f1f873e7acb609036
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 358.1 MB 5.3 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595728 sha256=5f9a13159ed2fde8154368a748856f64408db7e030f9b3d629e3239f61907bc6
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.0 MB 5.3 MB/s 
     |████████████████████████████████| 311 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |████████████████████████████████| 132 kB 42.3 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 8.5 MB 38.5 MB/s 
     |████████████████████████████████| 138 kB 50.7 MB/s 
     |███████████

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기

In [5]:
TRAIN = '/content/drive/MyDrive/DACON_MONTHLYNLI/train_data.csv'
TEST = '/content/drive/MyDrive/DACON_MONTHLYNLI/test_data.csv'
SS = '/content/drive/MyDrive/DACON_MONTHLYNLI/sample_submission.csv'

train = pd.read_csv(TRAIN)
test = pd.read_csv(TEST)
submission = pd.read_csv(SS)

In [8]:
max_len = 90
batch_size = 64    # Batch size 감소
warmup_ratio = 0.1
num_epochs = 6
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

config = {
    'max_len' : max_len,
    'batch_size' : batch_size,
    'warmup_ratio' : warmup_ratio,
    'num_epochs' : num_epochs, 
    'max_grad_norm' : max_grad_norm,
    'log_interval' : log_interval,
    'learning_rate' : learning_rate
}

print(config)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

{'max_len': 70, 'batch_size': 64, 'warmup_ratio': 0.1, 'num_epochs': 10, 'max_grad_norm': 1, 'log_interval': 200, 'learning_rate': 5e-05}


In [9]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir = ".cache")

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
print(pd.unique(train["label"]))

label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}

['contradiction' 'entailment' 'neutral']


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sen_idx, label_idx, bert_tokenizer, max_len,
                pad, pair, mode = "train"):
        self.mode = mode
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length = max_len, pad = pad, pair = pair)
        if self.mode == "train":
            self.sentence = [transform(i[sen_idx]) for i in dataset]   # pair=True를 위해 수정. [] 제거
            self.labels = [np.int32(i[label_idx]) for i in dataset]
            
        else:
            self.sentence =  [transform(i[sen_idx]) for i in dataset]   # pair=True를 위해 수정. [] 제거
        
    def __getitem__(self, i):
        if self.mode == 'train':
            return (self.sentence[i] +  (self.labels[i], ))
        else:
            return self.sentence[i]
    
    def __len__(self):
        return (len(self.sentence))

# Tokenizing
* Baseline 코드를 돌려 보던 중 Tokeninzing에서 이상한 점을 발견
* [CLS]와 [SEP]가 토큰으로 인식 되지 않고 분해되는 것을 발견함
* Fair를 활용

In [12]:
# [CLS], [SEP] 토큰 분해
text1 = train['premise'][0]
text2 = train['hypothesis'][0]
concat_text = "[CLS]" + text1 + "[SEP] " + text2 + "[SEP]"
tok(concat_text)

['▁[',
 '▁C',
 'LS',
 '▁',
 ']',
 '▁씨',
 '름',
 '은',
 '▁상',
 '고',
 '시대',
 '로부터',
 '▁전해',
 '져',
 '▁내려',
 '오는',
 '▁남자',
 '들의',
 '▁대표적인',
 '▁놀',
 '이',
 '로서',
 '▁',
 ',',
 '▁소',
 '년',
 '이나',
 '▁장',
 '정',
 '들이',
 '▁넓',
 '고',
 '▁평',
 '평',
 '한',
 '▁백',
 '사',
 '장',
 '이나',
 '▁마',
 '당',
 '에서',
 '▁모여',
 '▁서로',
 '▁힘',
 '과',
 '▁슬',
 '기를',
 '▁',
 '겨',
 '루',
 '는',
 '▁것이다',
 '▁',
 '.',
 '▁[',
 '▁S',
 'E',
 'P',
 '▁',
 ']',
 '▁씨',
 '름',
 '의',
 '▁여자',
 '들의',
 '▁놀',
 '이',
 '이다',
 '▁',
 '.',
 '▁[',
 '▁S',
 'E',
 'P',
 '▁',
 ']']

In [13]:
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad = True, pair = False)

# [CLS]와 [SEP]는 자동으로 붙여주는거 같음
transform(concat_text)

(array([  2, 702,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1], dtype=int32),
 array(3, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int32))

In [14]:
# Fair를 활용한 tokenizing
text1 = train['premise'][0]
text2 = train['hypothesis'][0]

# pair True 설정
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad = True, pair = True)
transform([text1, text2])

(array([   2, 3088, 6117, 7086, 2658, 5439, 6708, 6080, 4059, 7245, 1442,
        6965, 1423, 5939, 1678, 1504, 7096, 6081,  517,   46, 2822, 5712,
        7098, 3954, 7227, 5940, 1459, 5439, 4841, 7724, 7828, 2298, 6493,
        7178, 7098, 1907, 5804, 6903, 2064, 2720, 5211, 5468, 2948, 5573,
         517, 5411, 6095, 5760,  913,  517,   54,    3, 3088, 6117, 7095,
        3318, 5939, 1504, 7096, 7100,  517,   54,    3,    1,    1,    1,
           1,    1,    1,    1], dtype=int32),
 array(63, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0], dtype=int32))

In [15]:
# Fair를 활용
# train["premise_"] = "[CLS]" + train["premise"] + "[SEP]"
# train["hypothesis_"] = train["hypothesis"] + "[SEP]"

# test["premise_"] = "[CLS]" + test["premise"] + "[SEP]"
# test["hypothesis_"] = test["hypothesis"] + "[SEP]"

# train["text_sum"] = train.premise_ + " " + train.hypothesis_
# test["text_sum"] = test.premise_ + " " + test.hypothesis_

train_content = []
test_content = []

for idx in tqdm(train.index):
    train_content.append(list([[train.loc[idx, 'premise'], train.loc[idx, 'hypothesis']], label_dict[train.loc[idx, 'label']]]))
    
for idx in tqdm(test.index):
    test_content.append(list([[test.loc[idx, 'premise'], test.loc[idx, 'hypothesis']]]))
  
dataset_train = train_content[:20000]
dataset_valid = train_content[20000:]
dataset_test = test_content

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, True, mode = "train")
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, True, mode = "train")
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, True, mode = "test")

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 0)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size = batch_size, num_workers = 0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 0)

100%|██████████| 1666/1666 [00:00<00:00, 35272.76it/s]


# 모델 만들기
* Baseline 코드와 동일
* Kobert 공식 튜토리얼 문서에 따라 warmup scheduler 적용
* Batch size를 128로 동일하게 하고 싶었으나 메모리 문제로 64로 줄임(GPU 지원 받습니다.)

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=3, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

        
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# warup scheduler
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)   

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [18]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [19]:
import wandb
wandb.init(
        project="YBIGTA20 DACON NLI",
        config = config
)

best_loss = 100000

for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    train_loss = 0.0
    val_loss = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()    # scheduler 적용
        train_acc += calc_accuracy(out, label)
        train_loss += loss
    print("-"*20)
    print("epoch {} train acc {} train loss {}".format(e+1, train_acc / (batch_id+1), train_loss / (batch_id+1)))
    wandb.log({"Training MAE Loss": train_loss/(batch_id+1), "epoch":e+1})

    model.eval()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(valid_dataloader), total=len(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        valid_acc += calc_accuracy(out, label)
        val_loss += loss_fn(out, label)
    print("-"*20)
    print("epoch {} valid acc {} val loss {}".format(e+1, valid_acc / (batch_id+1), val_loss / (batch_id+1)))
    wandb.log({"Validation MAE Loss": val_loss/(batch_id+1), "epoch":e+1})

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


100%|██████████| 313/313 [07:34<00:00,  1.45s/it]


--------------------
epoch 1 train acc 0.4264676517571885 train loss 1.0638387203216553


  4%|▍         | 3/79 [00:01<00:42,  1.78it/s]


RuntimeError: ignored

# 추론 및 제출

In [ ]:
result = []
model.eval()
with torch.no_grad():
    for batch_id, (token_ids, valid_length, segment_ids) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        result.append(model(token_ids, valid_length, segment_ids))

100%|██████████| 27/27 [00:16<00:00,  1.66it/s]


In [ ]:
result_ = []
for i in result:
    for j in i:
        result_.append(int(torch.argmax(j)))
        
out = [list(label_dict.keys())[_] for _ in result_]

submission["label"] = out

submission.to_csv("kobert_baseline.csv", index = False)